In [1]:
# ============================================================
# EBM CREDIT SCORING - FULL THESIS PIPELINE
# ============================================================
# Tác giả: Gemini & User
# Mục tiêu: So sánh EBM Baseline vs. EBM Optimized (Optuna)
# ============================================================

import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.metrics import roc_auc_score
from optuna.visualization import plot_optimization_history, plot_param_importances

# Cấu hình hiển thị pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# ============================================================
# 1. HÀM TIỆN ÍCH (UTILS)
# ============================================================

def feature_engineering_final(df_in):
    """
    Tạo biến phái sinh (Feature Engineering) cho file EBM.
    Lưu ý: KHÔNG xử lý Outlier/Missing ở đây nữa vì EDA đã làm rồi.
    Chỉ thực hiện các phép tính toán học (Cộng trừ nhân chia, Log).
    """
    df = df_in.copy()
    
    # 1.1. Xử lý fillna tạm thời để tính toán Ratio (tránh chia cho NaN)
    # Lưu ý: EBM tự xử lý được NaN ở biến gốc, nhưng biến Ratio cần số cụ thể
    if 'INCOME' in df.columns:
        income_safe = df['INCOME'].fillna(0)
    else:
        income_safe = 0
        
    if 'CBAL' in df.columns:
        cbal_safe = df['CBAL'].fillna(0)
    else:
        cbal_safe = 0
        
    if 'AFLIMT_MAX' in df.columns:
        limit_safe = df['AFLIMT_MAX'].replace(0, 1).fillna(1)
    else:
        limit_safe = 1

    # 1.2. Tạo Ratios
    # DTI: Debt to Income
    df['RATIO_DTI'] = cbal_safe / (income_safe + 1)
    
    # Utilization: Dư nợ / Hạn mức
    df['RATIO_UTILIZATION'] = cbal_safe / limit_safe
    
    # Payment Burden (giả định trả 3% dư nợ)
    df['RATIO_PAYMENT_TO_INCOME'] = (cbal_safe * 0.03) / (income_safe + 1)
    
    # 1.3. Log Transformations (Giúp phân phối chuẩn hơn cho EBM)
    for col in ['INCOME', 'CBAL', 'BASE_AUM']:
        if col in df.columns:
            # Tự động xử lý số âm bằng cách clip tại 0 trước khi log
            df[f'{col}_LOG'] = np.log1p(df[col].clip(lower=0))

    # 1.4. Interactions (Tương tác biến thủ công - Domain Knowledge)
    # Ví dụ: DTI cao mà LTV cũng cao -> Rủi ro kép
    if 'LTV' in df.columns:
        df['DTI_x_LTV'] = df['RATIO_DTI'] * df['LTV']
        
    # Tiền gửi nhiều x Thu nhập cao -> Khách VIP (Rủi ro thấp)
    if 'N_AVG_DEPOSIT_12M' in df.columns and 'INCOME_LOG' in df.columns:
        df['DEPOSIT_x_INCOME'] = df['N_AVG_DEPOSIT_12M'] * df['INCOME_LOG']
        
    # 1.5. Clean Ratios (Cắt đuôi nhẹ cho các biến vừa tạo ra thôi)
    # Các biến gốc đã được cap ở EDA, nhưng biến mới tạo (như DTI) có thể bị vọt
    if 'RATIO_DTI' in df.columns:
        df['RATIO_DTI'] = df['RATIO_DTI'].clip(0, 10)
    if 'RATIO_UTILIZATION' in df.columns:
        df['RATIO_UTILIZATION'] = df['RATIO_UTILIZATION'].clip(0, 5)

    return df

def calculate_gini(model, X, y, label=""):
    """Tính Gini và in ra màn hình"""
    # EBM predict_proba trả về xác suất [P(0), P(1)]
    prob = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, prob)
    gini = 2 * auc - 1
    if label:
        print(f"   [{label}] AUC: {auc:.4f} | GINI: {gini:.4f}")
    return gini

# ============================================================
# 2. MAIN WORKFLOW
# ============================================================

print(">>> [1/6] LOADING CLEAN DATA (FROM EDA)...")

# Đường dẫn thư mục chứa file Parquet đã xuất từ EDA
data_path = r'C:\Users\PC\Documents\GitHub\Khoa-luan\EBM'

try:
    # Đọc 6 file riêng biệt
    X_train = pd.read_parquet(f'{data_path}/X_train.parquet')
    y_train = pd.read_parquet(f'{data_path}/y_train.parquet').squeeze() # squeeze để về Series

    X_val = pd.read_parquet(f'{data_path}/X_oos.parquet')
    y_val = pd.read_parquet(f'{data_path}/y_oos.parquet').squeeze()

    X_test = pd.read_parquet(f'{data_path}/X_oot.parquet')
    y_test = pd.read_parquet(f'{data_path}/y_oot.parquet').squeeze()
    
    print(f"   Train: {X_train.shape} | OOS: {X_val.shape} | OOT: {X_test.shape}")

except FileNotFoundError:
    print("LỖI: Không tìm thấy file Parquet. Hãy chạy file EDA trước để xuất dữ liệu.")
    exit()

print(">>> [2/6] FEATURE ENGINEERING (ON-THE-FLY)...")
# Áp dụng hàm tạo biến phái sinh cho cả 3 tập
X_train = feature_engineering_final(X_train)
X_val   = feature_engineering_final(X_val)
X_test  = feature_engineering_final(X_test)

# Cập nhật danh sách feature sau khi tạo thêm biến mới
feature_cols = X_train.columns.tolist()
print(f"   Số lượng biến đưa vào mô hình: {len(feature_cols)}")

# ------------------------------------------------------------
# CẤU HÌNH MONOTONICITY (Cập nhật theo biến thực tế có trong X)
# ------------------------------------------------------------
# Dictionary định nghĩa hướng rủi ro (Logic nghiệp vụ)
mono_constraints_dict = {
    # Tăng -> Risk Tăng (1)
    'MAX_DPD_12M_OBS': 1, 'AVG_OD_DPD_12M': 1, 'SUM_ALL_OD_12M': 1, 
    'CBAL_AVG': 1, 'LTV': 1, 'RATE_AVG': 1,
    'RATIO_DTI': 1, 'RATIO_UTILIZATION': 1, 'RATIO_PAYMENT_TO_INCOME': 1,
    'DTI_x_LTV': 1,

    # Tăng -> Risk Giảm (-1)
    'INCOME': -1, 'INCOME_LOG': -1,
    'BASE_AUM': -1, 'AUM_LOG': -1,
    'N_AVG_DEPOSIT_12M': -1, 'DEPOSIT_x_INCOME': -1,
    'SOHUUNHA': -1,
    
    # Không ràng buộc (0)
    'TUOI': 0, 'DURATION_MAX': 0
}

# Tạo list constraints khớp với thứ tự cột trong X_train
# Nếu biến không có trong dict, mặc định là 0
mono_constraints_list = [mono_constraints_dict.get(col, 0) for col in feature_cols]

# ============================================================
# 3. BASELINE MODEL
# ============================================================
print("\n>>> [3/6] TRAINING BASELINE MODEL...")

ebm_base = ExplainableBoostingClassifier(
    monotone_constraints=mono_constraints_list, 
    interactions=0,               
    learning_rate=1,    
    random_state=42,    
    n_jobs=-1,  
    early_stopping_rounds=10    # Dừng sớm nếu không cải thiện trên tập Validation (cần truyền eval_set)
)

# Fit mô hình (Có sử dụng tập Validation để Early Stopping)
ebm_base.fit(X_train, y_train) 
# Lưu ý: EBM mặc định sẽ tự tách một phần train ra làm val nội bộ nếu không đưa eval_set.
# Nếu muốn chuẩn chỉ dùng OOS làm val:
# ebm_base.fit(X_train, y_train, eval_set=[(X_val, y_val)]) 

# Đánh giá Baseline
print("   Kết quả Baseline:")
gini_train = calculate_gini(ebm_base, X_train, y_train, "TRAIN-Base")
gini_base_oos = calculate_gini(ebm_base, X_val, y_val, "OOS-Base")
gini_base_oot = calculate_gini(ebm_base, X_test, y_test, "OOT-Base")

# Hiển thị Global Explanation (Feature Importance)
show(ebm_base.explain_global()) # Uncomment nếu chạy trên Jupyter Notebook

c:\Users\PC\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> [1/6] LOADING CLEAN DATA (FROM EDA)...
   Train: (1137807, 37) | OOS: (300317, 37) | OOT: (302113, 37)
>>> [2/6] FEATURE ENGINEERING (ON-THE-FLY)...
   Số lượng biến đưa vào mô hình: 45

>>> [3/6] TRAINING BASELINE MODEL...


c:\Users\PC\miniconda3\Lib\site-packages\interpret\glassbox\_ebm\_ebm.py:871: UserWarning: Missing values detected. Our visualizations do not currently display missing values. To retain the glassbox nature of the model you need to either set the missing values to an extreme value like -1000 that will be visible on the graphs, or manually examine the missing value score in ebm.term_scores_[term_index][0]
  warn(


   Kết quả Baseline:
   [TRAIN-Base] AUC: 0.8714 | GINI: 0.7428
   [OOS-Base] AUC: 0.8774 | GINI: 0.7547
   [OOT-Base] AUC: 0.8755 | GINI: 0.7510


<!-- http://127.0.0.1:7001/1529063906608/ -->

In [2]:
feature_cols

['C_GIOITINH',
 'TTHONNHAN',
 'NHANVIENBIDV',
 'BASE_AUM',
 'TUOI',
 'INCOME',
 'CBAL',
 'AFLIMT_AVG',
 'LTV',
 'N_AVG_DEPOSIT_12M',
 'FLAG_SALARY_ACC',
 'FLAG_DEPOSIT',
 'UTILIZATION_RATE',
 'CNT_CREDIT_CARDS',
 'AMT_CASH_ADVANCE_12M',
 'PCT_PAYMENT_TO_BALANCE',
 'CNT_MIN_PAY_6M',
 'AVG_DAYS_PAST_DUE',
 'DTI_RATIO',
 'MOB',
 'CNT_OTHER_PRODUCTS',
 'LIMIT_TO_INCOME',
 'AMT_VAR_6M',
 'CBAL_SHORTTERM_LOAN',
 'CBAL_LONGTERM_LOAN',
 'HAS_LONGTERM_LOAN',
 'CNT_DPD_30PLUS_6M',
 'OCCUPATION_TYPE',
 'DURATION_MAX',
 'REMAINING_DURATION_MAX',
 'TIME_TO_OP_MAX',
 'RATE_AVG',
 'PURCOD_MAX',
 'MAX_DPD_12M',
 'AVG_OD_DPD_12M',
 'MAX_NHOMNOCIC',
 'N_AVG_OVERDUE_CBAL_12M',
 'RATIO_DTI',
 'RATIO_UTILIZATION',
 'RATIO_PAYMENT_TO_INCOME',
 'INCOME_LOG',
 'CBAL_LOG',
 'BASE_AUM_LOG',
 'DTI_x_LTV',
 'DEPOSIT_x_INCOME']

In [3]:
# # ============================================================
# # 4. OPTUNA TUNING – EBM OPTIMIZED
# # ============================================================
# from optuna.pruners import HyperbandPruner
# print("\n>>> [3/6] OPTIMIZING MODEL WITH OPTUNA...")

# def objective(trial):
#     params = {
#         "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
#         "interactions": trial.suggest_int("interactions", 0, 20),
#         "max_bins": trial.suggest_int("max_bins", 128, 512),
#         "outer_bags": trial.suggest_int("outer_bags", 4, 20),
#         "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 100),
#     }

#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#     gini_scores = []

#     for train_idx, val_idx in skf.split(X_train, y_train):
#         X_tr, X_va = X_train.iloc[train_idx], X_train.iloc[val_idx]
#         y_tr, y_va = y_train.iloc[train_idx], y_train.iloc[val_idx]

#         model = ExplainableBoostingClassifier(
#             monotone_constraints=mono_constraints_list,
#             random_state=42,
#             n_jobs=-1,
#             **params
#         )

#         model.fit(X_tr, y_tr)

#         prob = model.predict_proba(X_va)[:, 1]
#         auc = roc_auc_score(y_va, prob)
#         gini_scores.append(2*auc - 1)

#     return np.mean(gini_scores)

# study = optuna.create_study(direction="maximize",
#     pruner=optuna.pruners.MedianPruner(
#         n_startup_trials=5,
#         n_warmup_steps=1,
#         interval_steps=1

#     ))
# study.optimize(objective, n_trials=50)

# print("\n>>> BEST PARAMETERS FOUND:")
# print(study.best_params)
# print(f"Best Gini (OOS): {study.best_value:.4f}")

# # Train lại model optimized
# ebm_opt = ExplainableBoostingClassifier(
#     monotone_constraints=mono_constraints_list,
#     random_state=42,
#     n_jobs=-1,
#     **study.best_params
# )
# ebm_opt.fit(X_train, y_train)

# print("\n>>> [4/6] GINI – EBM OPTIMIZED")
# gini_opt_oos = calculate_gini(ebm_opt, X_val, y_val, "OOS-Optimized")
# gini_opt_oot = calculate_gini(ebm_opt, X_test, y_test, "OOT-Optimized")

# # Visualization
# fig1 = plot_optimization_history(study)
# fig2 = plot_param_importances(study)
# fig1.show()
# fig2.show()

# # Save model
# pickle.dump(ebm_opt, open("ebm_optimized.pkl", "wb"))

In [4]:
# import optuna.visualization as vis

# # 1. Vẽ Slice Plot cho tất cả các tham số
# fig_slice = vis.plot_slice(study)
# fig_slice.show()

# # 2. Hoặc vẽ Slice Plot cho một vài tham số quan trọng bạn muốn đưa vào bài viết
# fig_specific = vis.plot_slice(study, params=["learning_rate", "min_samples_leaf", "interactions"])
# fig_specific.show()

In [5]:
# # Train lại model optimized
# ebm_opt = ExplainableBoostingClassifier(
#     monotone_constraints=mono_constraints_list,
#     random_state=42,
#     n_jobs=-1,
#     learning_rate=0.041459457759190334, 
#     interactions=8, 
#     max_bins=325, 
#     outer_bags=11, 
#     min_samples_leaf=27
# )
# ebm_opt.fit(X_train, y_train)

# print("\n>>> [4/6] GINI – EBM OPTIMIZED")
# gini_opt_oos = calculate_gini(ebm_opt, X_val, y_val, "OOS-Optimized")
# gini_opt_oot = calculate_gini(ebm_opt, X_test, y_test, "OOT-Optimized")


In [6]:
# pickle.dump(ebm_opt, open("ebm_optimized.pkl", "wb"))

In [7]:
# # ============================================================
# # 5. MODEL EXPLAINABILITY
# # ============================================================

# print("\n>>> [5/6] EXPLAINABILITY REPORT...")

# # 5.1. GLOBAL FEATURE IMPORTANCE
# global_importance = ebm_opt.explain_global()

# print("\n>>> TOP FEATURES (Global Importance):")
# for i, (name, score) in enumerate(zip(
#         global_importance.data()['names'],
#         global_importance.data()['scores'])):
#     print(f"{i+1:2d}. {name}: {score:.4f}")

# # 5.2. HIỂN THỊ BẰNG TRÌNH DUYỆT (nếu chạy trong notebook thì hiện ngay)
# try:
#     show(global_importance)
# except:
#     print("   (Không thể render giao diện trực tiếp — vẫn tiếp tục.)")

# # 5.3. LOCAL EXPLANATION (giải thích 1 khách hàng bất kỳ)
# sample_index = 0 
# sample_X = X_test.iloc[[0]]  # vẫn là DataFrame 1 hàng
# sample_y = y_test.iloc[0]    # Series scalar, 1D

# local_exp = ebm_opt.explain_local(X_test, y_test)
#   # Series



# print("\n>>> LOCAL EXPLANATION SAMPLE:")
# show(local_exp)
# print(local_exp.data(0))


# try:
#     importances = ebm_opt.term_importances()
# except:
#     try:
#         importances = ebm_opt.feature_importances_
#     except:
#         raise ValueError("Không tìm thấy thuộc tính term_importances hoặc feature_importances.")

# terms = ebm_opt.term_names_

# print("\n>>> TOP INTERACTIONS:")
# for name, imp in sorted(zip(terms, importances), key=lambda x: x[1], reverse=True):
#     if "&" in name:  # interaction luôn có ký tự &
#         print(f"{name}: {imp:.4f}")


# # Lưu phần explain vào file
# pickle.dump(global_importance, open("ebm_global_exp.pkl", "wb"))
# pickle.dump(local_exp, open("ebm_local_exp.pkl", "wb"))

Global importance: 
- Giá trị tuyệt đối trung bình của điểm số (Average Absolute Score) mà biến đó đóng góp vào dự báo trên toàn bộ tập dữ liệu: Trung bình, biến này làm thay đổi dự đoán (log-odds) bao nhiêu đơn vị? Số càng lớn = Biến càng quan trọng.

Cách tính, ví dụ với biến SOHUUNHA:
- Bước 1: Tính Score cho từng người: Với mỗi khách hàng trong tập dữ liệu, mô hình xem khách hàng đó có nhà hay không (SOHUUNHA = 1 hoặc 0). Sau đó, nó tra cứu trong bảng hàm hình dạng xem giá trị đó tương ứng với bao nhiêu điểm (ví dụ: Có nhà thì $-0.8$ điểm, không nhà thì $+1.3$ điểm rủi ro).

- Bước 2: Lấy trị tuyệt đối: Vì mô hình muốn đo lường mức độ tác động (dù là làm tăng hay giảm rủi ro đều được coi là quan trọng), nên nó lấy trị tuyệt đối của tất cả các điểm số đó: $|-0.8| = 0.8$ và $|1.3| = 1.3$.

- Bước 3: Tính trung bình: Cộng tất cả các trị tuyệt đối này lại và chia cho tổng số khách hàng. Kết quả cuối cùng chính là con số bạn thấy.

Tác động của biến SOHUUNHA: 
- SOHUUNHA ≈ 0 (Không sở hữu nhà): * Giá trị Score nằm ở mức khoảng +0.8. Những khách hàng không có nhà riêng có xu hướng làm tăng xác suất của biến mục tiêu
- SOHUUNHA ≈ 1 (Có sở hữu nhà): * Giá trị Score giảm mạnh xuống mức khoảng -1.5. Việc sở hữu nhà đóng góp một giá trị âm rất lớn vào tổng điểm. Điều này làm giảm mạnh xác suất xảy ra biến mục tiêu

In [8]:
# # ============================================================
# # 6. FINAL REPORT – PSI, COMPARISON, SUMMARY
# # ============================================================

# print("\n>>> [6/6] FINAL MODEL REPORT...")

# # 6.1. Tính PSI của 2 biến chính giữa Train – OOT
# psi_report = {}
# for col in ["INCOME", "CBAL", "RATIO_DTI", "RATIO_UTILIZATION"]:
#     try:
#         psi_report[col] = calculate_psi(
#             X_train[col].values,
#             X_test[col].values
#         )
#     except:
#         psi_report[col] = None

# print("\n>>> PSI REPORT (Train → OOT):")
# for k,v in psi_report.items():
#     print(f"   {k}: {v:.4f}")

# # 6.2. So sánh Gini Baseline vs Optimized
# print("\n>>> GINI COMPARISON")
# print(f"Baseline – OOS: {gini_base_oos:.4f}")
# print(f"Optimized – OOS: {gini_opt_oos:.4f}")
# print(f"Baseline – OOT: {gini_base_oot:.4f}")
# print(f"Optimized – OOT: {gini_opt_oot:.4f}")

# # 6.3. Lưu bản summary ra file TXT
# # Best Hyperparameters:
# # {study.best_params}
# # """
# summary_text = f"""
# =============================
# EBM CREDIT SCORING SUMMARY
# =============================

# GINI – OOS:
#  - Baseline:  {gini_base_oos:.4f}
#  - Optimized: {gini_opt_oos:.4f}

# GINI – OOT:
#  - Baseline:  {gini_base_oot:.4f}
#  - Optimized: {gini_opt_oot:.4f}

# PSI (Train → OOT):
# {psi_report}

# """

# with open("model_summary.txt", "w") as f:
#     f.write(summary_text)

# print("\n>>> SUMMARY EXPORTED → model_summary.txt\n")


Mô hình rất ổn định khi chuyển từ TRAIN → OOT

In [9]:
# # 6.4. Tính PSI cho Score (Xác suất dự báo) - QUAN TRỌNG NHẤT
# prob_train = ebm_opt.predict_proba(X_train)[:, 1]
# prob_oot = ebm_opt.predict_proba(X_test)[:, 1]

# psi_score = calculate_psi(prob_train, prob_oot)
# print(f"\n>>> FINAL MODEL STABILITY (PSI Score): {psi_score:.4f}")
# if psi_score < 0.1:
#     print("    => Kết luận: Mô hình cực kỳ ổn định qua thời gian.")
# elif psi_score < 0.25:
#     print("    => Kết luận: Mô hình có sự biến động nhẹ, cần giám sát.")
# else:
#     print("    => Kết luận: CẢNH BÁO! Mô hình không ổn định, cần kiểm tra lại dữ liệu OOT.")

In [10]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# # ============================================================
# # 7. NÂNG CAO: MONOTONICITY, CONFUSION MATRIX & RATING
# # ============================================================
# print("\n>>> [7/7] ADVANCED ANALYTICS & RATING...")

# # 7.1. Kiểm tra tính đơn điệu (Monotonicity Check)
# # EBM lưu các hàm đóng góp trong explain_global. 
# # Bạn có thể vẽ lại để kiểm tra xem các biến ép đơn điệu có đúng đường thẳng/bậc thang không.
# def check_monotonicity(ebm_model, feature_name):
#     exp = ebm_model.explain_global(name=feature_name)
#     plt.figure(figsize=(8, 5))
#     plt.step(exp.data(0)['names'], exp.data(0)['scores'], where='post')
#     plt.title(f"Monotonicity Check: {feature_name}")
#     plt.xlabel(feature_name)
#     plt.ylabel("Logit Contribution")
#     plt.grid(True)
#     plt.show()

# # Chạy thử cho 2 biến tiêu biểu
# # check_monotonicity(ebm_opt, 'MAX_DPD_12M_OBS') 
# # check_monotonicity(ebm_opt, 'LTV')

# # 7.2. Ma trận nhầm lẫn tại điểm cắt (Cut-off)
# # Giả sử ngân hàng chọn Cut-off là 10% (PD > 0.1 là từ chối)
# threshold = 0.1
# y_prob_oot = ebm_opt.predict_proba(X_test)[:, 1]
# y_pred_oot = (y_prob_oot >= threshold).astype(int)

# print(f"\n>>> CONFUSION MATRIX (At Threshold {threshold}):")
# cm = confusion_matrix(y_test, y_pred_oot)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Good (0)', 'Bad (1)'])
# disp.plot(cmap='Blues')
# plt.title(f"Confusion Matrix at Cut-off {threshold}")
# plt.show()

# print(classification_report(y_test, y_pred_oot))

# # 7.3. Phân hạng khách hàng (Credit Rating)
# def assign_rating(pd):
#     if pd <= 0.01: return 'AAA (Rất an toàn)'
#     if pd <= 0.03: return 'AA'
#     if pd <= 0.05: return 'A'
#     if pd <= 0.10: return 'BBB'
#     if pd <= 0.20: return 'BB'
#     if pd <= 0.50: return 'B'
#     return 'C (Rủi ro cao - Từ chối)'

# # Tạo bảng kết quả cuối cùng cho tập Test
# df_result = pd.DataFrame({
#     'Actual': y_test.values,
#     'PD': y_prob_oot
# })

# df_result['Rating'] = df_result['PD'].apply(assign_rating)

# print("\n>>> THỐNG KÊ HẠNG KHÁCH HÀNG TRÊN TẬP OOT:")
# rating_dist = df_result['Rating'].value_counts().sort_index()
# print(rating_dist)

# # Tính tỷ lệ nợ xấu thực tế trên mỗi hạng (Để kiểm tra tính phân tách)
# bad_rate_by_rating = df_result.groupby('Rating')['Actual'].mean()
# print("\n>>> TỶ LỆ NỢ XẤU THỰC TẾ THEO HẠNG (Bad Rate by Grade):")
# print(bad_rate_by_rating)

In [11]:
# # ============================================================
# # 8. CREDIT SCORECARD CALCULATION (300 - 850)
# # ============================================================
# print("\n>>> [8/7] CONVERTING PD TO CREDIT SCORE (300-850)...")

# def calculate_credit_score(pd, pdo=20, base_score=600, base_odds=50):
#     """
#     Chuyển đổi PD sang Credit Score dựa trên công thức chuẩn:
#     Score = Offset + Factor * ln(odds)
#     - pdo: Points to Double the Odds (thường là 20 hoặc 50)
#     - base_score: Điểm cơ sở (ví dụ 600 điểm)
#     - base_odds: Tỷ lệ odds tương ứng với điểm cơ sở (ví dụ 50:1)
#     """
#     # Tránh chia cho 0 hoặc log(0)
#     pd = np.clip(pd, 0.0001, 0.9999)
    
#     factor = pdo / np.log(2)
#     offset = base_score - factor * np.log(base_odds)
    
#     odds = (1 - pd) / pd
#     score = offset + factor * np.log(odds)
    
#     return np.round(score).astype(int)

# # Áp dụng cho tập Test (OOT)
# df_result['Credit_Score'] = calculate_credit_score(df_result['PD'])

# # Giới hạn điểm trong khoảng 300 - 850 (như chuẩn FICO)
# df_result['Credit_Score'] = df_result['Credit_Score'].clip(300, 850)

# print("\n>>> THỐNG KÊ ĐIỂM TÍN DỤNG TRÊN TẬP OOT:")
# print(df_result['Credit_Score'].describe())

# # Vẽ biểu đồ phân phối điểm số
# plt.figure(figsize=(10, 6))
# plt.hist(df_result['Credit_Score'], bins=50, color='skyblue', edgecolor='black')
# plt.axvline(df_result['Credit_Score'].mean(), color='red', linestyle='dashed', linewidth=2, label='Average Score')
# plt.title('Distribution of Credit Scores (OOT Set)')
# plt.xlabel('Credit Score')
# plt.ylabel('Number of Customers')
# plt.legend()
# plt.show()

# # Kiểm tra sự tương quan giữa Hạng và Điểm trung bình
# print("\n>>> ĐIỂM TRUNG BÌNH THEO HẠNG:")
# print(df_result.groupby('Rating')['Credit_Score'].mean().sort_values(ascending=False))